<a href="https://colab.research.google.com/github/marcoscleytton/simple-ml-regression/blob/main/rf_20features_gridsearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import de bibliotecas

import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

In [10]:
# criar a função do bags com mais features  informativas e intancias
def create_bags (num_bags= 100, num_instances_per_bag= 20):
    x, y = [],[]
    num_positive_bags = num_bags // 2
    num_negative_bags = num_bags - num_positive_bags


    for _ in range(num_positive_bags):
        # aqui vamos aumentar o numero de features e torna mais informativa
        bag = make_classification(n_samples=num_instances_per_bag,n_features=20,n_informative=15)
        x.append(bag[0])
        y.append(1) # Assign label 1 for positive bags

    for _ in range(num_negative_bags):
        bag = make_classification(n_samples=num_instances_per_bag,n_features=20,n_informative=15)
        x.append(bag[0])
        y.append(0) # Assign label 0 for negative bags


    return x, y

# gerar as bags
x, y = create_bags()

# flatten dos dados para treinameno do modelo
x_flat = np.array([instance for bag in x for instance in bag])
y_flat = np.array([y[i] for i, bag in enumerate(x) for instance in bag])


# dividir o conjunto de dados em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x_flat, y_flat, test_size=0.3, stratify=y_flat)

# definir parametros para otimizalção com mais regularização
param_grid = {
    'n_estimators': [100], # mantemos 100 instimadores
    'max_depth': [10],    # restrição de profundidade
    'min_samples_split': [5, 10], # aumentar o criterio para divisão
    'min_samples_leaf': [4, 6] # regularização adiocional
}

In [12]:
# gerar as bags
x, y = create_bags()

# flatten dos dados para treinameno do modelo
x_flat = np.array([instance for bag in x for instance in bag])
y_flat = np.array([y[i] for i, bag in enumerate(x) for instance in bag])


# dividir o conjunto de dados em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x_flat, y_flat, test_size=0.3, stratify=y_flat)

# definir parametros para otimizalção com mais regularização
param_grid = {
    'n_estimators': [100], # mantemos 100 instimadores
    'max_depth': [10],    # restrição de profundidade
    'min_samples_split': [5, 10], # aumentar o criterio para divisão
    'min_samples_leaf': [4, 6] # regularização adiocional
}

In [14]:
# configurar o gridsearchCV

grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

# melhor modelo encontrado pelo gridsearchCV

best_model = grid_search.best_estimator_

# fazer previsão com melhor modelo
y_pred = best_model.predict(x_test)

In [16]:
# avaliar  a acuracia do modelo random forest otimizado

accuracy_best_rf = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo random forest otimizado: {accuracy_best_rf:.2f}')

# avaliar o modelo otimizado com roc & auc

roc_score_best_rf = roc_auc_score(y_test, best_model.predict_proba(x_test)[:, 1])
print(f'ROC AUC do modelo random forest otimizado: {roc_score_best_rf:.2f}')

# testar o gradient boosting

from sklearn.ensemble import GradientBoostingClassifier

# treinar o modelo gradient boosting

model_gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=10, random_state=42)
model_gb.fit(x_train, y_train)

# fazendo predições com radient boosting

y_pred_gb = model_gb.predict(x_test)

# avaliar a acuracia e roc auc do gradient boosting

accuracy_gb = accuracy_score(y_test, y_pred_gb)
roc_score_gb = roc_auc_score(y_test, model_gb.predict_proba(x_test)[:, 1])

print(f'Acurácia do modelo Gradient Boosting: {accuracy_gb:.2f}')
print(f'ROC AUC do modelo Gradient Boosting: {roc_score_gb:.2f}')

Acurácia do modelo random forest otimizado: 0.57
ROC AUC do modelo random forest otimizado: 0.57
Acurácia do modelo Gradient Boosting: 0.57
ROC AUC do modelo Gradient Boosting: 0.59


In [17]:
# testar o gradient

import xgboost as xgb

In [18]:
# treinar modelo xboosting

model_xgb = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=10, random_state=42)
model_xgb.fit(x_train, y_train)

# fazer previsão com xboosting

y_pred_xgb = model_xgb.predict(x_test)

# avaliar a acuracia roc auc do xboosting

accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
roc_score_xgb = roc_auc_score(y_test, model_xgb.predict_proba(x_test)[:, 1])

print(f'Acurácia do modelo XGBoost: {accuracy_xgb:.2f}')
print(f'ROC AUC do modelo XGBoost: {roc_score_xgb:.2f}')

Acurácia do modelo XGBoost: 0.56
ROC AUC do modelo XGBoost: 0.58
